# Orifice Flow Rate Calculation

A differential pressure of 2.5 psi is measured across taps located 1 diameter upstream and 0.5 diamter downstream from the inlet face of a 2.000" ID orifice plate assembled in a 3" Schedule 80 steel pipe carrying water at 60 degF.
Find the flowrate

In [1]:
import fluids
from fluids.units import *
from fluids.control_valve import size_control_valve_l
from thermo.chemical import Chemical
import math

get the nearest internal pipe diameter

In [2]:
deltaP = 2.5*u.psi
d1 = 2*u.inch
#d2 = nearest_pipe(Do=3*u.inch, schedule = '80s')
d2 = nearest_pipe(Do=3*u.inch, schedule='80')[1].to(u.inch)

In [3]:
beta = d1/d2
beta

<Quantity(0.689655172413793, 'dimensionless')>

According to Crane 410, 4-2, Limits of Use.  For orifice plates with corner taps or 1D-1/2D taps, for beta>0.56, Re >=5000.  Since flow in pipe is not given let's assume an Re of 5000.

In [4]:
Re = 100000
L1 = 1 # Ratio of upstream tap distance to pipe diameter
L2_prime = 0.47 #Ratio of downstream tap distance to pipe diameter, less than 0.5 due to orifice plate thinkness
M2_prime = 2*L2_prime/(1-beta)
J = (19000*beta/Re)**0.8

In [5]:
Cd = 0.5961 + 0.0261*beta**2 - 0.216*beta**8 + 0.000521*(beta*10**6/Re)**0.7 \
    +(0.0188+0.0063*J)*beta**3.5*(10**6/Re)**0.3 + (0.043+0.080*math.exp(-10*L1)-0.123*math.exp(-7*L1))\
    *(1-0.11*J)*(beta**4/(1-beta**4)) - 0.031*(M2_prime-0.8*M2_prime**1.1)*beta**1.3 + 0.011*(0.75-beta)*(2.8-d2.magnitude)

Cd

<Quantity(0.6164124716452033, 'dimensionless')>

In [6]:
C = Cd/math.sqrt(1-beta**4)

C

<Quantity(0.7007485816011257, 'dimensionless')>

Get the density of water assume water if P=2atm

In [7]:
T = (60*u.degF).to(u.degK)
P = (2*14.7*u.psi).to(u.Pa)

water = Chemical('water', P=P.magnitude , T=T.magnitude)
S = water.SG #specific gravity
rho = water.rho #density
rho = (rho*u.kilogram/(u.meter**3)).to(u.lb/u.cubic_feet)
mu = water.mu #viscosity
mu = mu*1000 #cP

rho, mu


(<Quantity(62.47022612300394, 'pound / cubic_foot')>, 1.1348540881678713)

In [8]:
Q = 234.6*d1.magnitude**2*C*math.sqrt(deltaP.magnitude/rho.magnitude)
Q = Q*u.gallon/u.minute
Q

<Quantity(131.54783103899248, 'gallon / minute')>

Recalculate Re using this Q

In [9]:
Re = 50.66*Q.magnitude*rho.magnitude/(d2.magnitude*mu)
Re

126498.07914577075

In [10]:
Cd = 0.5961 + 0.0261*beta**2 - 0.216*beta**8 + 0.000521*(beta*10**6/Re)**0.7 \
    +(0.0188+0.0063*J)*beta**3.5*(10**6/Re)**0.3 + (0.043+0.080*math.exp(-10*L1)-0.123*math.exp(-7*L1))\
    *(1-0.11*J)*(beta**4/(1-beta**4)) - 0.031*(M2_prime-0.8*M2_prime**1.1)*beta**1.3 + 0.011*(0.75-beta)*(2.8-d2.magnitude)
C = Cd/math.sqrt(1-beta**4)\

Cd, C


(<Quantity(0.6153654399192205, 'dimensionless')>,
 <Quantity(0.6995582974478614, 'dimensionless')>)

In [11]:
Q = 234.6*d1.magnitude**2*C*math.sqrt(deltaP.magnitude/rho.magnitude)
Q = Q*u.gallon/u.minute
Q

<Quantity(131.32438527999537, 'gallon / minute')>

The solution in Crane 410 uses the appendix figures.  The solution is about the same.